## Agent Supervisor Example

- collaboration 예제의 경우 에이전트 출력에 따라 자동으로 라우팅됨
- LLM을 사용하여 다양한 에이전트를 오케스트레이션 가능
- 작업을 위임하는데 도움을 주는 에이전트 감독관과 함께 에이전트 그룹 생성 가능

In [ ]:
import dotenv
dotenv.load_dotenv()

## Create Tools
- 검색 엔진을 사용하는 웹 리서치 에이전트와 그래프를 생성하는 에이전트 생성

In [1]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

python_repl_tool = PythonREPLTool()

## Helper Utilities

그래프에서 노드를 생성하는데 필요한 헬처 함수 정의
- 이 함수는 에이전트의 응답을 사람의 메세지로 변환하는 작업 처리

In [3]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["messages"][-1].content, name=name)]}